In [438]:
import pandas as pd
import numpy as np
import glob
import re
import traceback
pd.set_option('display.max_columns', 500)

# Summary

<i>The overall objective of this program is to map raw data extracted from PDFs to different financial categories such as Tangible Assets, Intangible Assets, Current Liabilities, etc.In essence, the program reads each raw file in a given directory and produces a mapped file of each company's balance sheet.</i>

<ol>
    <li>Read each file and filter for balance sheet tab</li>
    <li>Filter each file by section and find and map items to their respective lists via regular expression</li>
    <li>return a mapped dataframe/object for each section</li>
    <li>Combined each mapped section to single Dataframe</li>
    <li>Modify master dataframe by only showing columns that have not been restated and removing old columns that have been restated.</li>
    <li>Produce Total Assets and Liabilities rows in the master dataframe</li>
</ol>

In [460]:
def generate_company_list():
    companies = []
    unique_list = []
    file_list = glob.glob("C:/Users/curt.beck/OneDrive/Financial_Mapping/Ares/*.xlsx") # change filepath
    for file in file_list:
        match_obj = re.search("[A-Za-z]+\_\d+|[A-Za-z]+\s+[\&A-Za-z\s\_\d]+", file)
        if match_obj is not None:
            split_str = match_obj.group().split('_')
            split_str_item = split_str[0]
            companies.append(split_str_item)
    
    for company in companies:
        if company not in unique_list:
            unique_list.append(company)
    no_spaces = [company.strip() for company in unique_list]
    clean_list = [company for company in no_spaces if company]
    return clean_list

In [461]:
generate_company_list()

['Avetta',
 'Elmica Parent',
 'Flinn Scientific',
 'FMSystems Group',
 'Novipax Buyer']

In [309]:
#filters for the balance sheet tab
def filter_sheets(sheet_list, statement_type):
    if statement_type == 'Income Statement':
        filtered_list = [sheet for sheet in sheet_list if re.search('IS\s+\-\s+\d{1}', sheet)]
        if len(filtered_list) > 0:
            return filtered_list[0]
        else:
            return [] 
    elif statement_type == 'Balance Sheet':
        filtered_list = [sheet for sheet in sheet_list if re.findall('BS\s+\-\s+\d{1}', sheet)]
        if len(filtered_list) > 1:
            return filtered_list
        elif len(filtered_list) == 1:
            return filtered_list[0]
        else:
            return []
    elif statement_type == 'Cash Flow':
        filtered_list = [sheet for sheet in sheet_list if re.search('CF\s+\-\s+\d{1}', sheet)]
        if len(filtered_list) > 0:
            return filtered_list[0]
        else:
            return []

In [3]:
# Creates a date array for each file to be used in the mapped file
def create_date_period_array(file, sheet_name):
    df = pd.read_excel(file, sheet_name='Metadata')
    filtered_df = df.loc[df['Sheet name'] == sheet_name]
    filtered_df_period = filtered_df.filter(regex="Value\s+column\s+\d{1}\s+period\s+coverage", axis=1).dropna(axis=1)
    filtered_df_date = filtered_df.filter(regex="Value\s+column\s+\d{1}\s+date", axis=1).dropna(axis=1)
    return list(filtered_df_date.values[0]), list(filtered_df_period.values[0])

Each template section will be generated by function, which returns a dataframew

In [4]:
#Filters raw file for the current asset section of the balance sheet.
def filter_current_asset_section(df):
    copy_df = df
    item_list = copy_df[copy_df.columns[0]].tolist()
    
    start_match = df.iloc[:,0].str.extract(r'(Cash.*)').dropna().values[0][0]
    start_index = item_list.index(start_match)
    
    end_match = df.iloc[:,0].str.extract(r'([Tt]otal\s+[Cc]urrent\s+[Aa]ssets)').dropna().values[0][0]
    end_index = item_list.index(end_match)
    
    copy_df.set_index(df.columns[0], inplace=True)
    filtered_df = copy_df.iloc[start_index:end_index]
    return filtered_df

In [445]:
def filter_non_current_asset_section(df):
    copy_df = df
    item_list = copy_df[copy_df.columns[0]].tolist()
    
    start_match = df.iloc[:,0].str.extract(r'(.*?[Tt]otal\s+[Cc]urrent\s+[Aa]ssets|TOTAL\s+CURRENT\s+ASSETS)').dropna().values[0][0]
    start_index = item_list.index(start_match)+1
    
    end_match = df.iloc[:,0].str.extract(r'([Tt]otal\s+[Aa]ssets|[Tt]otal\s+[lL]ong\-term.*|TOTAL\s+ASSETS)').dropna().values[0][0]
    end_index = item_list.index(end_match)
    
    copy_df.set_index(df.columns[0], inplace=True)
    filtered_df = copy_df.iloc[start_index:end_index].dropna()
    #filtered_df.dropna(inplace=True)
    return filtered_df

In [446]:
df = pd.read_excel("C:\\Users\\curt.beck\\Downloads\\Cognaize\\Novipax Buyer_202102.xlsx", sheet_name="BS - 2")
filtered_df = filter_non_current_asset_section(df)
filtered_df

,MONTH RESULTS,YEAR RESULTS
Unnamed: 0,,
Land and Buildings,10179,10179
Building Improvements,1824,1824
Computer Software and Equipment,178,178
Furniture and Fixtures,24,24
Machinery and Equipment,13094,13094
Construction in Progress,3252,3252
"Total Fixed Assets, Net",28551,28551
Intangibles,-13,-13
Goodwill**,219721,219721


In [310]:
#If multiple items are mapped to a single category, this function sums the values of those items, which will then be mapped to the appropriate category
def sum_multiple_rows(rows, df, counter):
    flat_rows = list(rows.flatten())
    rows_df = df[df[df.columns[0]].isin(flat_rows)]
    row_sum = sum(list(rows_df.iloc[:, counter+1].values))
    return row_sum

In [410]:
# Returns the mapped section of the current asset portion of the balance sheet
def current_assets(file, sheet_name, date_array):
    cash_list = []
    inventory_list = []
    other_list_final_list = []
    acct_rec_other_combined_final_list = []
    curr_asset_labels = []
    totals_list = []
    #for date in date_array:
    for i in range(0, len(date_array)):
        other_list = []
        acct_rec_list = []
        prepaid_list = []
        inventory_list_first = []
        df = pd.read_excel(file, sheet_name=sheet_name)
        df[df.columns[0]] = df[df.columns[0]].str.lstrip()
        df[df.columns[0]] = df[df.columns[0]].str.rstrip()
        filtered_df = filter_current_asset_section(df)
        filtered_df.reset_index(inplace=True)
        
         # find cash & cash evquivalents
        if filtered_df.iloc[:,0].str.extract(r'(Cash.*)').dropna().empty:
            print("No Cash")
        else:
            cash_label = filtered_df.iloc[:,0].str.extract(r'(Cash.*)').dropna().values[0][0]
            filtered_df.set_index(filtered_df.columns[0], inplace=True)
            cash_val = filtered_df.loc[cash_label].values[i]
            cash_list.append(cash_val)
            curr_asset_labels.append(cash_label)
            filtered_df.reset_index(inplace=True)
    #find inventory
        if filtered_df.iloc[:,0].str.extract(r'(Inven)').dropna().empty:
            print("No Inventory")
        else:
            inven_label = filtered_df.iloc[:,0].str.extract(r'(Inven.*)').dropna().values[0][0]
            filtered_df.set_index(filtered_df.columns[0], inplace=True)
            inven_val = filtered_df.loc[inven_label].values[i]
            inventory_list_first.append(inven_val)
            curr_asset_labels.append(inven_label)
            filtered_df.reset_index(inplace=True)
        
        if filtered_df.iloc[:,0].str.extract(r'(Accounts\s+[rR]ec.*)').dropna().empty:
            print("No Accounts Receievable")
        elif len(filtered_df.iloc[:,0].str.extract(r'(Accounts\s+[rR]ec.*)').dropna().values)>1:
            acct_rece_label = filtered_df.iloc[:,0].str.extract(r'(Accounts\s+[rR]ec.*)').dropna().values
            acct_rece_label = list(acct_rece_label.flatten())
            acct_rec_rows = filtered_df.iloc[:,0].str.extract(r'(Accounts\s+[rR]ec.*)').dropna().values
            acct_rows_sum = sum_multiple_rows(acct_rec_rows, filtered_df, i)
            acct_rec_list.append(acct_rows_sum)
            curr_asset_labels = curr_asset_labels + acct_rece_label
            #flattened_acct_rows = list(acct_rec_rows.flatten())
            #acct_rec_df = filtered_df[filtered_df[filtered_df.columns[0]].isin(flattened_acct_rows)]
            #acct_rec_list.append(sum(list(acct_rec_df.iloc[:, i+1].values)))
        else:
            acct_rece_label = filtered_df.iloc[:,0].str.extract(r'(Accounts\s+[rR]ec.*)').dropna().values[0][0]
            filtered_df.set_index(filtered_df.columns[0], inplace=True)
            acc_rece_val = filtered_df.loc[acct_rece_label].values[i]
            acct_rec_list.append(acc_rece_val)
            curr_asset_labels.append(acct_rece_label)
            filtered_df.reset_index(inplace=True)
        #if filtered_df.iloc[:,0].str.extract(r'(Prepaid.*)').dropna().empty:
        #    print("No Prepaid Expenses")
        #elif len(filtered_df.iloc[:,0].str.extract(r'(Prepaid.*)').dropna().values)>1:
        #    prepaid_rows = filtered_df.iloc[:,0].str.extract(r'(Prepaid.*)').dropna().values
        #    prepaid_row_sum = sum_multiple_rows(prepaid_rows, filtered_df, i)
        #    prepaid_list.append(prepaid_row_sum)
        #else:
        #    prepaid_label = filtered_df.iloc[:,0].str.extract(r'(Prepaid.*)').dropna().values[0][0]
        #    filtered_df.set_index(filtered_df.columns[0], inplace=True)
        #    prepaid_val = filtered_df.loc[prepaid_label].values[i]
        #    prepaid_list.append(prepaid_val)
        #    curr_asset_labels.append(prepaid_label)
        #    filtered_df.reset_index(inplace=True)
    
        
        other_asset_df = filtered_df[~filtered_df[filtered_df.columns[0]].isin(curr_asset_labels)]
        other_list.append(sum(other_asset_df.iloc[:, i+1].values))
        other_list_final_list.append(sum(other_list))
        acct_rec_other_combined_list = acct_rec_list
        acct_rec_other_combined_final_list.append(sum(acct_rec_other_combined_list))
        inventory_list.append(sum(inventory_list_first))
        
        #temp_list = [[cash_val], inventory_list_first, acct_rec_other_combined_list, other_list]
        #temp_list_flat = [item for sublist in temp_list for item in sublist]
        #temp_list_flattened = temp_list.flatten()
        #totals = sum(temp_list_flat)
        #totals_list.append(totals)
    data = [cash_list, inventory_list, acct_rec_other_combined_final_list, other_list_final_list]
    final_df = pd.DataFrame(data=data, columns=date_array, index=['Cash and Cash Equivalents', 'Inventories', 'Trade and Other Receivables', 'Other Current Assets'])
    final_df.loc['Total Current Assets'] = final_df.sum()
    return final_df
        

        
    
        
        
        

In [411]:
current_assets("C:\\Users\\curt.beck\\Downloads\\Cognaize\\Avetta_202012.xlsx", "BS - 1", test)

No Inventory
No Inventory


,2020-12-31,2019-12-31
Cash and Cash Equivalents,33123.0,19415.0
Inventories,0.0,0.0
Trade and Other Receivables,2329.0,2406.0
Other Current Assets,6800.0,6346.0
Total Current Assets,42252.0,28167.0


In [412]:
# Returns a mapped section of the non current asset portion of the balance sheet
def non_current_assets(file, sheet_name, date_array):
    intangible_list_final = []
    tangible_list = []
    other_list = []
    non_curr_labels = []
    totals_list = []
    for i in range(0, len(date_array)):
        intangible_list_first = []
        tangible_list_first = []
        goodwill_list = []
        other_list_first = []
        df = pd.read_excel(file, sheet_name=sheet_name)
        df[df.columns[0]] = df[df.columns[0]].str.lstrip()
        df[df.columns[0]] = df[df.columns[0]].str.rstrip()
        filtered_df = filter_non_current_asset_section(df)
        filtered_df.reset_index(inplace=True)
        #print(filtered_df.columns)
        if filtered_df.iloc[:,0].str.extract(r'(Intangible.*|[Oo]perating\s+lease.*|[Ss]oftware.*)').dropna().empty:
            print("No Intangibles")
        
        elif len(filtered_df.iloc[:,0].str.extract(r'(Intangible.*|[Oo]perating\s+[lL]ease.*|[Ss]oftware.*)').dropna()) > 1:
            intan_label = filtered_df.iloc[:,0].str.extract(r'(Intangible.*|[Oo]perating\s+[lL]ease.*|[Ss]oftware.*)').dropna().values
            intan_label = list(intan_label.flatten())
            intan_label_rows = filtered_df.iloc[:,0].str.extract(r'(Intangible.*|[Oo]perating\s+[lL]ease.*|[Ss]oftware.*)').dropna().values
            intan_rows_sum = sum_multiple_rows(intan_label_rows, filtered_df, i)
            intangible_list_first.append(intan_rows_sum)
            non_curr_labels = non_curr_labels + intan_label
            #intangible_list_final.append(intangible_list_first[0])
        else:
            intan_label = filtered_df.iloc[:,0].str.extract(r'(Intangible.*|[Oo]perating\s+[Ll]ease.*|[Ss]oftware.*)').dropna().values[0][0]
            print(intan_label)
            filtered_df.set_index(filtered_df.columns[0], inplace=True)
            intan_val = filtered_df.loc[intan_label].values[i]
            intangible_list_first.append(intan_val)
            non_curr_labels.append(intan_label)
            filtered_df.reset_index(inplace=True)
            
        if filtered_df.iloc[:,0].str.extract(r'(Goodwill.*)').dropna().empty:
            print("No Goodwill")
        else:
            intan_label = filtered_df.iloc[:,0].str.extract(r'(Goodwill.*)').dropna().values[0][0]
            filtered_df.set_index(filtered_df.columns[0], inplace=True)
            intan_val = filtered_df.loc[intan_label].values[i]
            goodwill_list.append(intan_val)
            non_curr_labels.append(intan_label)
            filtered_df.reset_index(inplace=True)
        
        intangible_list_final_vals = intangible_list_first + goodwill_list
        #intangible_list_first.append(sum(intangible_list_final_vals))
        intangible_list_final.append(sum(intangible_list_final_vals))
        
        
        if filtered_df.iloc[:,0].str.extract(r'(Property.*|Capital\s+[Ll]ease.*|Less\s+[Aa]cc.*)').dropna().empty:
            print("No Property...")
        elif len(filtered_df.iloc[:,0].str.extract(r'(Property.*|Capital\s+[Ll]ease.*|Less\s+[Aa]cc.*)'))>1:
            tan_label = filtered_df.iloc[:,0].str.extract(r'(Property.*|Capital\s+[Ll]ease.*|Less\s+[Aa]cc.*)').dropna().values
            tan_label = list(tan_label.flatten())
            tan_label_rows = filtered_df.iloc[:,0].str.extract(r'(Property.*|Capital\s+[Ll]ease.*|Less\s+[Aa]cc.*)').dropna().values
            tan_rows_sum = sum_multiple_rows(tan_label_rows, filtered_df, i)
            tangible_list_first.append(tan_rows_sum)
            non_curr_labels = non_curr_labels + tan_label
            tangible_list.append(tangible_list_first[0])
        else:
            tan_label = filtered_df.iloc[:,0].str.extract(r'(Property.*|Capital\s+[Ll]ease.*|Less\s+[Aa]cc.*)').dropna().values[0][0]
            filtered_df.set_index(filtered_df.columns[0], inplace=True)
            tan_val = filtered_df.loc[tan_label].values[i]
            tangible_list_first.append(tan_val)
            non_curr_labels.append(tan_label)
            filtered_df.reset_index(inplace=True)
            tangible_list.append(tangible_list_first)

        #tangible_list = list(tangible_list)
        #non_curr_asset_label = filtered_df.iloc[:,0].str.extract(r'(Total\s+[a|A]ssets|TOTAL\s+ASSETS)').dropna().values[0][0]
        #filtered_df.set_index(df.columns[0], inplace=True)
        #index_list = filtered_df.index.tolist()
        #total_curr_asset_index = index_list.index(non_curr_asset_label)
        #property_index = index_list.index(tan_label)
        #filtered_df.reset_index(inplace=True)

        #if filtered_df.iloc[:,0].str.extract(r'(Total\s+long\-term\s+assets)').dropna().empty:
        #    non_curr_asset_df = filtered_df.iloc[property_index:total_curr_asset_index]
         #   non_curr_asset_df.reset_index(inplace=True)
          #  other_non_curr_asset_df = non_curr_asset_df[~non_curr_asset_df[non_curr_asset_df.columns[0]].isin(non_curr_labels)]
        #else:
         #   non_curr_total_label = filtered_df.iloc[:,0].str.extract(r'(Total\s+long\-term\s+assets)').dropna().values[0][0]
          #  non_curr_total_index = index_list.index(non_curr_total_label)
           # non_curr_asset_df = filtered_df.iloc[property_index:non_curr_total_index]
           # non_curr_asset_df.reset_index(inplace=True)
        other_non_curr_asset_df = filtered_df[~filtered_df[filtered_df.columns[0]].isin(non_curr_labels)]
        other_list.append(sum((list(other_non_curr_asset_df.iloc[:, i+1].values[0:]))))
        other_list_first.append(sum((list(other_non_curr_asset_df.iloc[:, i+1].values[0:]))))
        
        #master_list = [intangible_list_first, tangible_list_first, other_list_first]
        #master_list_flat = [item for sublist in master_list for item in sublist]
        #totals_list.append(sum(master_list_flat))
        #print(intangible_list_first)
    data = [intangible_list_final, tangible_list, other_list]
    final_df = pd.DataFrame(data=data, columns=date_array, index=['Intangible Assets', 'Tangible Assets', 'Other Long-Term Assets'])
    final_df.loc['Total Non-Current Assets'] = final_df.sum()
    return final_df
        

In [413]:
non_current_assets("C:\\Users\\curt.beck\\Downloads\\Cognaize\\Flinn Scientific_202105.xlsx", "BS - 1", test)


Intangible Assets
Intangible Assets


,2020-12-31,2019-12-31
Intangible Assets,108711,119708
Tangible Assets,13834,14188
Other Long-Term Assets,122547,134808
Total Non-Current Assets,245092,268704


In [417]:
# Filters the file for the current liability section of the balance sheet
def filter_current_liabilities_section(df):
    copy_df = df
    item_list = copy_df[copy_df.columns[0]].tolist()
    
    if df.iloc[:,0].str.extract(r'(Current\s+Liabilities\:?|CURRENT\s+LIABILITIES\:?)').dropna().empty:
        start_match = df.iloc[:,0].str.extract(r'([Aa]ccounts\s+[Pp]ay.*)').dropna().values[0][0]
        start_index = item_list.index(start_match)
    else:
        start_match = df.iloc[:,0].str.extract(r'([Aa]ccounts\s+[Pp]ay.*)').dropna().values[0][0]
        start_index = item_list.index(start_match)
    
    end_match = df.iloc[:,0].str.extract(r'([Tt]otal\s+[Cc]urrent\s+[Ll]iabilities)').dropna().values[0][0]
    end_index = item_list.index(end_match)
    
    copy_df.set_index(df.columns[0], inplace=True)
    filtered_df = copy_df.iloc[start_index:end_index]
    return filtered_df

In [450]:
# filters the file for non current liability section of the balance sheet
def filter_non_current_liabilities_section(df):
    copy_df = df
    item_list = copy_df[copy_df.columns[0]].tolist()
    start_match = copy_df.iloc[:,0].str.extract(r'([Tt]otal\s+[Cc]urrent\s+[Ll]iabilities)').dropna().values[0][0]
    start_index = item_list.index(start_match)+1
    
    
    end_match = copy_df.iloc[:,0].str.extract(r'([Tt]otal\s+[Ll]iabilities|Total\s+Long\s+Term\s+Liabilities|[Tt]otal\s+[Ll]iab[a-z]+|TOTAL\s+LIAB[A-Z]+)').dropna().values[0][0]
    end_index = item_list.index(end_match)
    
    #end_match_list = copy_df.iloc[:,0].str.extract(r'([Tt]otal\s+long.*)').dropna().values.tolist()
    #if len(end_match_list)
    if len(copy_df.iloc[:,0].str.extract(r'([Tt]otal\s+[Ll]ong.*)').dropna().values.tolist()) > 0:
        filtered_df = copy_df.iloc[start_index:end_index-1].dropna()
    else:
        filtered_df = copy_df.iloc[start_index:end_index].dropna()
    filtered_df.set_index(copy_df.columns[0], inplace=True)
    return filtered_df

In [452]:
df = pd.read_excel("C:\\Users\\curt.beck\\Downloads\\Cognaize\\Novipax Buyer_202102.xlsx", sheet_name="BS - 2")
filter_non_current_liabilities_section(df)

,MONTH RESULTS,YEAR RESULTS
Unnamed: 0,,
"Accrued Comp, Net of Current",0,0
"Long Term Debt,Net of Curr Port*",103301,103301


In [419]:
# Returns a mapped section of the current liabilities portion of the balance sheet
def current_liabilities(file, sheet_name, date_array):
    trade_other_rec_list_final = []
    curr_tax_liab_list_final = []
    short_debt_list_final = []
    other_liab_list = []
    curr_liab_labels = []
    totals_list = []
    
    for i in range(0, len(date_array)):
        trade_other_rec_list_first = []
        curr_tax_liab_list_first = []
        short_debt_list_first = []
        other_liab_list_first = []
        
        df = pd.read_excel(file, sheet_name=sheet_name)
        df[df.columns[0]] = df[df.columns[0]].str.lstrip()
        df[df.columns[0]] = df[df.columns[0]].str.rstrip()
        filtered_df = filter_current_liabilities_section(df)
        filtered_df.reset_index(inplace=True)
        
        if filtered_df.iloc[:,0].str.extract(r'([Aa]ccounts\s+[Pp]ay.*)').dropna().empty:
            print("No Accounts Payable")
        elif len(filtered_df.iloc[:,0].str.extract(r'([Aa]ccounts\s+[Pp]ay.*)').dropna().values) > 1:
            acct_pay_labels = filtered_df.iloc[:,0].str.extract(r'([Aa]ccounts\s+[Pp]ay.*)').dropna().values
            acct_pay_labels = list(acct_pay_labels.flatten())
            acct_pay_rows = filtered_df.iloc[:,0].str.extract(r'([Aa]ccounts\s+[Pp]ay.*)').dropna().values
            acct_pay_rows_sum = sum_multiple_rows(acct_pay_rows, filtered_df, i)
            trade_other_rec_list_first.append(acct_pay_rows_sum)
            curr_liab_labels = curr_liab_labels + acct_pay_labels
            trade_other_rec_list_final.append(trade_other_rec_list_first[0])
        else:
            acct_pay_label = filtered_df.iloc[:,0].str.extract(r'([Aa]ccounts\s+[Pp]ay.*)').dropna().values[0][0]
            filtered_df.set_index(filtered_df.columns[0], inplace=True)
            acct_pay_val = filtered_df.loc[acct_pay_label].values[i]
            trade_other_rec_list_first.append(acct_pay_val)
            curr_liab_labels.append(acct_pay_label)
            filtered_df.reset_index(inplace=True)
        
        #if not filtered_df.iloc[:,0].str.extract(r'([Aa]ccrued\s+[Ll]iabilities\:$)').dropna().empty:
        #    not_accrued_exp_array = filtered_df.iloc[:,0].str.extract('(^Current.*)').dropna().values
        #    not_accrued_exp_array_flat = list(not_accrued_exp_array.flatten())
        #    accrued_exp_label_df = filtered_df[~filtered_df[filtered_df.columns[0]].isin(not_accrued_exp_array_flat)]
        #    accrued_exp_label = list(accrued_exp_label_df[accrued_exp_label_df.columns[0]])
        #    accrued_exp_rows = accrued_exp_label_df[[accrued_exp_label_df.columns[0]]].values
        #    accrued_exp_row_sum = sum_multiple_rows(accrued_exp_rows, accrued_exp_label_df, i)
        #    curr_liab_labels = curr_liab_labels + accrued_exp_label
        #elif len(filtered_df.iloc[:,0].str.extract(r'([Aa]ccrued\s+([Ee]xp.*|[Ll]iabilites))').dropna())> 1:
        #    accrued_exp_label = filtered_df.iloc[:,0].str.extract(r'([Aa]ccrued\s+([Ee]xp.*|[Ll]iab.*))').dropna().values
        #    accrued_exp_rows = filtered_df.iloc[:,0].str.extract(r'([Aa]ccrued\s+([Ee]xp.*|[Ll]iab.*))').dropna().values
        #    accrued_exp_label = list(accrued_exp_label.flatten())
        #    accrued_exp_row_sum = sum_multiple_rows(accrued_exp_rows, filtered_df, i)
        #    trade_other_rec_list_first.append(accrued_exp_row_sum)
        #    curr_liab_labels = curr_liab_labels + accrued_exp_label
        #    trade_other_rec_list_final.append(trade_other_rec_list_first[0])
        #else:
        #    accrued_exp_label = filtered_df.iloc[:,0].str.extract(r'([Aa]ccrued\s+([Ee]xp.*|[Ll]iabilities))').dropna().values[0][0]
        #    filtered_df.set_index(filtered_df.columns[0], inplace=True)
        #    accrued_exp_val = filtered_df.loc[accrued_exp_label].values[i]
        #    trade_other_rec_list_first.append(accrued_exp_val)
        #    curr_liab_labels.append(accrued_exp_label)
        #    filtered_df.reset_index(inplace=True)
        trade_other_rec_list_final.append(sum(trade_other_rec_list_first))
        
        if filtered_df.iloc[:,0].str.extract(r'(Current.*[Dd]ebt|[Ll]ine\s+of.*)').dropna().empty:
            print("Not Short term debt")
        elif len(filtered_df.iloc[:,0].str.extract(r'(Current.*[Dd]ebt|[Ll]ine\s+of.*)').dropna().values) > 1:
            print(filtered_df.iloc[:,0].str.extract(r'(Current.*[Dd]ebt|[Ll]ine\s+of.*)').dropna().values)
            curr_debt_label = filtered_df.iloc[:,0].str.extract(r'(Current.*[Dd]ebt|[Ll]ine\s+of.*)').dropna().values
            curr_debt_rows = filtered_df.iloc[:,0].str.extract(r'(Current.*[Dd]ebt|[Ll]ine\s+of.*)').dropna().values
            curr_debt_label = list(curr_debt_label.flatten())
            curr_debt_sum = sum_multiple_rows(curr_debt_rows, filtered_df, i)
            short_debt_list_first.append(curr_debt_sum)
            short_debt_list_final.append(curr_debt_sum)
            curr_liab_labels = curr_liab_labels + curr_debt_label
        else:
            curr_debt_label = filtered_df.iloc[:,0].str.extract(r'(Current.*[Dd]ebt|[Ll]ine\s+of.*)').dropna().values[0][0]
            filtered_df.set_index(filtered_df.columns[0], inplace=True)
            curr_debt_val = filtered_df.loc[curr_debt_label].values[i]
            short_debt_list_first.append(curr_debt_val)
            curr_liab_labels.append(curr_debt_label)
            filtered_df.reset_index(inplace=True)
            short_debt_list_final.append(sum(short_debt_list_first))
        
        if filtered_df.iloc[:,0].str.extract(r'(^[Tt]axe?s?.*)').dropna().empty:
            print("No Taxes")
        else:
            curr_tax_label = filtered_df.iloc[:,0].str.extract(r'(^[Tt]axe?s?.*)').dropna().values[0][0]
            filtered_df.set_index(filtered_df.columns[0], inplace=True)
            curr_tax_val = filtered_df.loc[curr_tax_label].values[i]
            curr_tax_liab_list_first.append(curr_tax_val)
            curr_liab_labels.append(curr_tax_label)
            filtered_df.reset_index(inplace=True)
            curr_tax_liab_list_final.append(sum(curr_tax_liab_list_first))
        
        other_curr_liab_df = filtered_df[~filtered_df[filtered_df.columns[0]].isin(curr_liab_labels)]
        other_liab_list_first.append(sum((list(other_curr_liab_df.iloc[:, i+1].values[0:]))))
        other_liab_list.append(sum((list(other_curr_liab_df.iloc[:, i+1].values[0:]))))
        
        #temp_master_list = [trade_other_rec_list_first, curr_tax_liab_list_first, short_debt_list_first, other_liab_list_first]
        
        #temp_master_list_flat = [item for sublist in temp_master_list for item in sublist]
        #totals_list.append(sum(temp_master_list_flat))
        #print(temp_master_list_flat)
    #print(other_curr_liab_df)
    data = [trade_other_rec_list_final, curr_tax_liab_list_final, short_debt_list_final, other_liab_list]
    final_df = pd.DataFrame(data=data, columns=date_array, index=['Trade and Other Payables', 'Current Tax Liabilities', 'Short-Term Debt', 'Other Current Liabilities'])
    final_df.loc['Total Current Liabilities'] = final_df.sum()
    return final_df
        


In [504]:
current_liabilities("C:\\Users\\curt.beck\\OneDrive\\Financial_Mapping\\Ares\\Flinn Scientific_202009.xlsx", "BS - 1", test)

Not Short term debt
No Taxes
Not Short term debt
No Taxes


,2020-12-31,2019-12-31
Trade and Other Payables,3189.0,2539.0
Current Tax Liabilities,NaN,NaN
Short-Term Debt,NaN,NaN
Other Current Liabilities,NaN,NaN
Total Current Liabilities,3189.0,2539.0


In [421]:
# Returns mapped section of the non current liabilities portion of the balance sheet
def non_current_liabilities(file, sheet_name, date_array):
    long_debt_list_final = []
    provisions_list_final = []
    other_long_debt_final = []
    non_curr_liab_labels = []
    totals_list = []
    for i in range(0, len(date_array)):
        long_debt_list_first = []
        provisions_list_first = []
        other_long_debt_first = []
        
        df = pd.read_excel(file, sheet_name=sheet_name)
        df[df.columns[0]] = df[df.columns[0]].str.lstrip()
        df[df.columns[0]] = df[df.columns[0]].str.rstrip()
        filtered_df = filter_non_current_liabilities_section(df)
        filtered_df.reset_index(inplace=True)
        
        if len(filtered_df.iloc[:,0].str.extract(r'([Tt]erm.*|[Ll]ong.*|[Cc]ap.*|[Nn]ote.*|Revolving.*|Delayed.*|Operating\s+[lL]ease.*)').dropna().values) > 1:
            long_debt_label = filtered_df.iloc[:,0].str.extract(r'([Tt]erm.*|[Ll]ong.*|[Cc]ap.*|[Nn]ote.*|Revolving.*|Delayed.*|Operating\s+[lL]ease.*)').dropna().values
            long_debt_rows = filtered_df.iloc[:,0].str.extract(r'([Tt]erm.*|[Ll]ong.*|[Cc]ap.*|[Nn]ote.*|Revolving.*|Delayed.*|Operating\s+[lL]ease.*)').dropna().values
            long_debt_label = list(long_debt_label.flatten())
            long_debt_rows_sum = sum_multiple_rows(long_debt_rows, filtered_df, i)
            long_debt_list_first.append(long_debt_rows_sum)
            non_curr_liab_labels = non_curr_liab_labels + long_debt_label
            long_debt_list_final.append(long_debt_list_first[0])
        else:
            long_debt_label = filtered_df.iloc[:,0].str.extract(r'([Tt]erm.*|[Ll]ong.*|[Cc]ap.*|[Nn]ote.*|Revolving.*|Delayed.*|Operating\s+[lL]ease.*)').dropna().values[0][0] 
            filtered_df.set_index(filtered_df.columns[0], inplace=True)
            long_debt_val = filtered_df.loc[long_debt_label].values[i]
            long_debt_list_first.append(long_debt_val)
            non_curr_liab_labels.append(long_debt_label)
            filtered_df.reset_index(inplace=True)
            long_debt_list_final.append(sum(long_debt_list_first))
        
        
        other_non_curr_liab_df = filtered_df[~filtered_df[filtered_df.columns[0]].isin(non_curr_liab_labels)]
        
        other_long_debt_final.append(sum(list(other_non_curr_liab_df.iloc[:, i+1].values[0:])))
        other_long_debt_first.append(sum(list(other_non_curr_liab_df.iloc[:, i+1].values[0:])))
        #temp_master_list = [long_debt_list_first, provisions_list_first, other_long_debt_first]
        #temp_master_list_flat = [item for sublist in temp_master_list for item in sublist]
        #totals_list.append(sum(temp_master_list_flat))
    data = [long_debt_list_final, provisions_list_final, other_long_debt_final]
    final_df = pd.DataFrame(data=data, columns=date_array, index=['Long-Term Debt', 'Provisions', 'Other Long-Term Liabilities'])
    final_df.loc['Total Non-Current Liabilities'] = final_df.sum()
    return final_df

In [514]:
non_current_liabilities("C:\\Users\\curt.beck\\OneDrive\\Financial_Mapping\\Ares\\Novipax Buyer_202102.xlsx", "BS - 2", test)

,2020-12-31,2019-12-31
Long-Term Debt,103301.0,103301.0
Provisions,NaN,NaN
Other Long-Term Liabilities,0.0,0.0
Total Non-Current Liabilities,103301.0,103301.0


In [454]:
def total_equity_liabilities(file, sheet_name, date_array):
    equity_list = []
    equity_liab_list = []
    
    for i in range(0, len(date_array)):
        df = pd.read_excel(file, sheet_name=sheet_name)
        df[df.columns[0]] = df[df.columns[0]].str.lstrip()
        #df.reset_index(inplace=True)
        equity_label = df.iloc[:,0].str.extract(r'([Tt]otal.*[Ee]quity|TOTAL\s+EQU[A-Z]+)').dropna().values[0][0]
        df.set_index(df.columns[0], inplace=True)
        equity_val = df.loc[equity_label].values[i]
        equity_list.append(equity_val)
        df.reset_index(inplace=True)

        equity_liab_label = df.iloc[:,0].str.extract(r'([Tt]otal\s+[Ll]iab.*[Ee]quity|TOTAL\s+LIABILITIES\s+\&\s+EQ[A-Z]+)').dropna().values[0][0]
        df.set_index(df.columns[0], inplace=True)
        equity_liab_val = df.loc[equity_liab_label].values[i]
        equity_liab_list.append(equity_liab_val)
        #df.reset_index(inplace=True)
        
    data = [equity_list, equity_liab_list]
    final_df = pd.DataFrame(data=data, columns=date_array, index=['Equity', 'Total Liabilities and Equity'])
    return final_df

In [457]:
total_equity_liabilities("C:\\Users\\curt.beck\\Downloads\\Cognaize\\Novipax Buyer_202102.xlsx", "BS - 2", test)

,2020-12-31,2019-12-31
Equity,147531,147531
Total Liabilities and Equity,279047,279047


In [315]:
# Removes old columns that have been restated, keeping the restated ones and all of the columns that did not have restatements
def drop_old_data(df):
    exception_df = pd.DataFrame()
    unique_list = []
    col_list = df.columns
    for col in col_list:
        initial_col = col
        if df[[col]].shape[-1] > 1 and col not in unique_list:
            unique_list.append(col)
    
    for col in unique_list:
        temp_df = df[[col]].iloc[:, -1]
        exception_df = pd.concat([exception_df, temp_df], axis=1)
    non_dupe_df = df.loc[:, ~df.columns.isin(unique_list)]
    final_non_dupe_df = pd.concat([non_dupe_df, exception_df], axis=1)
    sorted_cols = np.sort(final_non_dupe_df.columns, axis=None)
    final_sorted_df = final_non_dupe_df[sorted_cols]
    return final_sorted_df

In [316]:
# Create total asset and liability rows in the master dataframe and returns and final dataframe to be exported to excel
def create_total_asset_total_liab_rows(df):
    total_curr_assets = df.loc['Total Current Assets'].values
    total_non_curr_assets = df.loc['Total Non-Current Assets'].values
    total_assets = np.add(total_curr_assets, total_non_curr_assets)
    
    total_curr_liab = df.loc['Total Current Liabilities'].values
    total_non_curr_liab = df.loc['Total Non-Current Liabilities'].values
    total_liab = np.add(total_curr_liab, total_non_curr_liab)
    
    df.loc['Total Assets'] = total_assets
    df.loc['Total Liabilities'] = total_liab
    return df

In [467]:
create_total_asset_total_liab_rows(reinstated_df)

,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-30,2021-11-30,2021-12-31,2022-01-31,2022-02-28
Intangible Assets,175342.0,219708.0,168322.0,168322.0,168423.0,168423.0,166943.0,166753.0,161188.0,160998.0,159518.0,145136.0,155501.0,153974.0
Tangible Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25900.0,NaN,NaN
Other Long-Term Assets,41441.0,60331.0,42914.0,42936.0,43093.0,43109.0,42419.0,42687.0,42430.0,42145.0,41914.0,171524.0,41695.0,41273.0
Total Non-Current Assets,216783.0,280039.0,211236.0,211258.0,211516.0,211532.0,209362.0,209440.0,203618.0,203143.0,201432.0,342560.0,197196.0,195247.0
Cash and Cash Equivalents,2734.0,3418.0,2617.0,2966.0,1613.0,1878.0,2951.0,1638.0,1813.0,4026.0,4749.0,3043.0,1266.0,4580.0
Inventories,9100.0,8164.0,9312.0,10551.0,11137.0,10585.0,10538.0,9465.0,9970.0,10099.0,9720.0,10649.0,10184.0,10816.0
Trade and Other Receivables,11899.0,14228.0,14138.0,12868.0,12397.0,13619.0,13126.0,13456.0,13883.0,14594.0,12600.0,10902.0,12610.0,13343.0
Other Current Assets,3184.0,1749.0,1659.0,2108.0,2133.0,2243.0,1995.0,1776.0,1643.0,1509.0,1767.0,14947.0,1395.0,948.0
Total Current Assets,26917.0,27559.0,27726.0,28493.0,27280.0,28325.0,28610.0,26335.0,27309.0,30228.0,28836.0,39541.0,25455.0,29687.0
Trade and Other Payables,11518.0,13114.0,13524.0,16316.0,14914.0,14321.0,14850.0,11118.0,10404.0,11154.0,10427.0,8944.0,10672.0,12907.0


# Main Script

<ol>
    <li>Read each file and filter for balance sheet tab</li>
    <li>Filter each file by section and find and map items to their respective lists via regular expression</li>
    <li>return a mapped dataframe/object for each section</li>
    <li>Combined each mapped section to single Dataframe</li>
    <li>Modify master dataframe by only showing columns that have not been restated and removing old columns that have been restated.</li>
    <li>Produce Total Assets and Liabilities rows in the master dataframe</li>
</ol>

In [516]:
companies = generate_company_list() # generates a list of companies to loop over and produce templates for
for company in companies:
    file_list = glob.glob(f"C:/Users/curt.beck/OneDrive/Financial_Mapping/Ares/{company}*.xlsx") # Change file path directory; Only has been tested for Avetta
    master_df_non_curr_asset = pd.DataFrame()
    master_df_curr_asset = pd.DataFrame()
    master_df_curr_liab = pd.DataFrame()
    master_df_non_curr_liab = pd.DataFrame()
    master_df_eq_liab_df = pd.DataFrame()
    final_master_df = pd.DataFrame()
    statement_type = 'Balance Sheet'
    for file in file_list:
        try:
            sheet_list = pd.ExcelFile(file).sheet_names
            sheet_name = filter_sheets(sheet_list, statement_type)
            if len(sheet_name) > 0:
                date_array, period_type_array = create_date_period_array(file, sheet_name)
                non_curr_asset_df = non_current_assets(file, sheet_name, date_array)
                curr_asset_df = current_assets(file, sheet_name, date_array)
                curr_liab_df = current_liabilities(file, sheet_name, date_array)
                non_curr_liab_df = non_current_liabilities(file, sheet_name, date_array)
                eq_liab_df = total_equity_liabilities(file, sheet_name, date_array)
        
                master_df_non_curr_asset = pd.concat([master_df_non_curr_asset, non_curr_asset_df], axis=1)
                master_df_curr_asset = pd.concat([master_df_curr_asset, curr_asset_df], axis=1)
                master_df_curr_liab = pd.concat([master_df_curr_liab, curr_liab_df], axis=1)
                master_df_non_curr_liab = pd.concat([master_df_non_curr_liab, non_curr_liab_df], axis=1)
                master_df_eq_liab_df = pd.concat([master_df_eq_liab_df, eq_liab_df], axis=1)
        #curr_liab_df = current_liabilities(file, sheet_name, date_array)
        #master_df_curr_liab = pd.concat([master_df_curr_liab, curr_liab_df], axis=1)
                final_master_df = pd.concat([master_df_non_curr_asset, 
                                     master_df_curr_asset, 
                                     master_df_curr_liab,
                                    master_df_non_curr_liab,
                                    master_df_eq_liab_df], axis=0)
        
                np_date_array = final_master_df.columns
                sorted_date_array = np.sort(np_date_array, axis=None)
        
                final_master_sorted_df = final_master_df[sorted_date_array]
                reinstated_df = drop_old_data(final_master_sorted_df)
                #reinstated_df.reset_index(inplace=True)
                #reinstated_df.set_index(reinstated_df.columns[0], inplace=True)
                final_reinstated_df = create_total_asset_total_liab_rows(reinstated_df)
                final_reinstated_df.to_excel(f"C:\\Users\\curt.beck\\OneDrive\Financial_Mapping\\Ares Output\\{company}_mapped_BS.xlsx")
            else:
                print(f"{file} does not have {statement_type}")
                pass
        except:
            print(final_master_df[sorted_date_array])
            print(f"There was an issue with {file}")
            traceback.print_exc()
            break

Intangible assets, net
Intangible assets, net
No Inventory
No Inventory
No Inventory
No Inventory
[['Current portion of long-term debt']
 ['Line of credit']]
No Taxes
[['Current portion of long-term debt']
 ['Line of credit']]
No Taxes
No Inventory
No Inventory
[['Current portion of long-term debt']
 ['Line of credit']]
No Taxes
[['Current portion of long-term debt']
 ['Line of credit']]
No Taxes
No Inventory
No Inventory
[['Current portion of long-term debt']
 ['Line of credit']]
No Taxes
[['Current portion of long-term debt']
 ['Line of credit']]
No Taxes
No Inventory
No Inventory
[['Current portion of long-term debt']
 ['Line of credit']]
No Taxes
[['Current portion of long-term debt']
 ['Line of credit']]
No Taxes
No Inventory
No Inventory
[['Current portion of long-term debt']
 ['Line of credit']]
No Taxes
[['Current portion of long-term debt']
 ['Line of credit']]
No Taxes
No Inventory
No Inventory
[['Current portion of long-term debt']
 ['Line of credit']]
No Taxes
[['Current po